In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import network_maps

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
plt.rcParams["figure.dpi"] = 200
x=3
y=3
fig_size = (x*y, y)

In [ ]:
data_dir = 'data/2025-05-06-twamp/'

list_of_files = os.listdir(data_dir)
list_of_files.sort()
num_files = len(list_of_files)
print('Number of files: {}'.format(num_files))

selected_columns = [
    'identity',
    'timestamp', # added by twamp logger
    #'@timestamp', # added by mqtt logger

    'outbound.min',
    'outbound.max',
    'outbound.avg',
    'outbound.jitter',
    'outbound.loss',

    'inbound.min',
    'inbound.max',
    'inbound.avg',
    'inbound.jitter',
    'inbound.loss',

    'roundtrip.min',
    'roundtrip.max',
    'roundtrip.avg',
    'roundtrip.jitter',
    'roundtrip.loss',
]

df_list = []
for file in list_of_files:
    # Skipping the files we're not using
    if file[-5:] != ".gzip": # .gzip
        continue
    temp_df = pd.read_parquet(data_dir+file, columns=selected_columns)
    df_list.append(temp_df)
df = pd.concat(df_list)
df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df['identity'].unique()

In [ ]:
df['identity'].value_counts()

In [ ]:
# Infer objects, then convert dtypes
df = df.infer_objects().convert_dtypes()

for column in df.columns:
        try:
            df[column] = pd.to_numeric(df[column])
        except (ValueError, TypeError):
            pass  # Skip columns that cannot be converted

# Rename timestamp from Python and keep it for future use
# It is unreliable if a lot of messages come at the same time due to congestion
df['timestamp_python'] = pd.to_datetime(df.pop('timestamp'))
df["timestamp_python"] = (
    df["timestamp_python"]
    .dt.tz_localize(None)        # 06:34:55.448743  (drop +02:00, keep clock time)
    + pd.Timedelta(hours=2)      # 08:34:55.448743
)
df = df.sort_values(by=['timestamp_python'])
df = df.set_index('timestamp_python', drop=False)

df.info(verbose=True, show_counts=True, memory_usage='deep')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.keys()

In [ ]:
df['identity'].unique()

In [ ]:
# #df = df.between_time('15:00', '21:00')
# start_date = '2025-01-22 09:00:00'
# end_date = '2025-01-22 18:00:00'
# mask = (df['timestamp_python'] > start_date) & (df['timestamp_python'] <= end_date)
# df = df.loc[mask] 

In [ ]:
df.query("identity == 'ADDIX-D2'").plot(y=['outbound.avg', 'inbound.avg', 'roundtrip.avg'], figsize=fig_size, ylim=(0, 300))

In [ ]:
df.query("identity == 'ADDIX-DTAG'").plot(y=['outbound.avg', 'inbound.avg', 'roundtrip.avg'], figsize=fig_size, ylim=(0, 300))

In [ ]:
df.query("identity == 'Starlink'").plot(y=['outbound.avg', 'inbound.avg', 'roundtrip.avg'], figsize=fig_size, ylim=(0, 300))

In [ ]:
df.query("identity == 'CAU-0C'").plot(y=['outbound.avg', 'inbound.avg', 'roundtrip.avg'], figsize=fig_size, ylim=(0, 300))

In [ ]:
df.query("identity == 'CAU-4329'").plot(y=['outbound.avg', 'inbound.avg', 'roundtrip.avg'], figsize=fig_size, ylim=(0, 300))

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

Y_LIM = (0, 500)

df.query("identity == 'ADDIX-D2'")['roundtrip.avg'].rename('D2-RTT').plot(ax=ax, color='r', ylim=Y_LIM)
df.query("identity == 'ADDIX-DTAG'")['roundtrip.avg'].rename('DTAG-RTT').plot(ax=ax, color='g', ylim=Y_LIM)
#df.query("identity == 'CAU-4329'")['roundtrip.avg'].rename('4329-RTT').plot(ax=ax, color='b', ylim=Y_LIM)
#df.query("identity == 'CAU-0C'")['roundtrip.avg'].rename('0C-RTT').plot(ax=ax, color='m', ylim=Y_LIM)
df.query("identity == 'Starlink'")['roundtrip.avg'].rename('Starlink-RTT').plot(ax=ax, color='c', ylim=Y_LIM)

ax.axhline(df.query("identity == 'ADDIX-D2'")['roundtrip.avg'].mean(), color='r', alpha=0.7, label='D2 Mean')
ax.axhline(df.query("identity == 'ADDIX-DTAG'")['roundtrip.avg'].mean(), color='g', alpha=0.7, label='DTAG Mean')
#ax.axhline(df.query("identity == 'CAU-4329'")['roundtrip.avg'].mean(), color='b', alpha=0.7, label='4329 Mean')
#ax.axhline(df.query("identity == 'CAU-0C'")['roundtrip.avg'].mean(), color='m', alpha=0.7, label='0C Mean')
ax.axhline(df.query("identity == 'Starlink'")['roundtrip.avg'].mean(), color='c', alpha=0.7, label='Starlink Mean')

ax.set_title('RTT over time')
ax.set_ylabel('RTT [ms]')
ax.grid(True, which='both', linestyle='--', alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
df_D2 = df.query("identity == 'ADDIX-D2'")
df_DTAG = df.query("identity == 'ADDIX-DTAG'")
df_Starlink = df.query("identity == 'Starlink'")
df_0C = df.query("identity == 'CAU-0C'")
df_4329 = df.query("identity == 'CAU-4329'")

In [ ]:
df_D2.info()

In [ ]:
df_DTAG.info()

In [ ]:
df_Starlink.info()

In [ ]:
df_0C.info()

In [ ]:
df_4329.info()

In [ ]:
df_D2.describe()

In [ ]:
df_DTAG.describe()

In [ ]:
df_Starlink.describe()

In [ ]:
df_0C.describe()

In [ ]:
df_4329.describe()

In [ ]:
fig, ax = plt.subplots(figsize=(40, 10))

df_D2['inbound.avg'].rename('D2-RTT').plot(ax=ax, ylim=(0, 1000))
df_DTAG['inbound.avg'].rename('DTAG-RTT').plot(ax=ax, ylim=(0, 1000))
df_Starlink['inbound.avg'].rename('Starlink-RTT').plot(ax=ax, ylim=(0, 1000))
df_0C['inbound.avg'].rename('0C-RTT').plot(ax=ax, ylim=(0, 1000))
df_4329['inbound.avg'].rename('4329-RTT').plot(ax=ax, ylim=(0, 1000))

ax.set_title('inbound over time')
ax.set_ylabel('inbound [ms]')
ax.grid(True, which='both', linestyle='--', alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(40, 10))

df_D2['inbound.jitter'].rename('D2-RTT').plot(ax=ax, ylim=(0, 1000))
df_DTAG['inbound.jitter'].rename('DTAG-RTT').plot(ax=ax, ylim=(0, 1000))
df_Starlink['inbound.jitter'].rename('Starlink-RTT').plot(ax=ax, ylim=(0, 1000))
df_0C['inbound.jitter'].rename('0C-RTT').plot(ax=ax, ylim=(0, 1000))
df_4329['inbound.jitter'].rename('4329-RTT').plot(ax=ax, ylim=(0, 1000))

ax.set_title('inbound jitter over time')
ax.set_ylabel('inbound jitter[ms]')
ax.grid(True, which='both', linestyle='--', alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(40, 10))

df_D2['outbound.avg'].rename('D2-RTT').plot(ax=ax, ylim=(0, 1000))
df_DTAG['outbound.avg'].rename('DTAG-RTT').plot(ax=ax, ylim=(0, 1000))
df_Starlink['outbound.avg'].rename('Starlink-RTT').plot(ax=ax, ylim=(0, 1000))
df_0C['outbound.avg'].rename('0C-RTT').plot(ax=ax, ylim=(0, 1000))
df_4329['outbound.avg'].rename('4329-RTT').plot(ax=ax, ylim=(0, 1000))

ax.set_title('outbound over time')
ax.set_ylabel('outbound [ms]')
ax.grid(True, which='both', linestyle='--', alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(40, 10))

df_D2['outbound.jitter'].rename('D2-RTT').plot(ax=ax, ylim=(0, 1000))
df_DTAG['outbound.jitter'].rename('DTAG-RTT').plot(ax=ax, ylim=(0, 1000))
df_Starlink['outbound.jitter'].rename('Starlink-RTT').plot(ax=ax, ylim=(0, 1000))
df_0C['outbound.jitter'].rename('0C-RTT').plot(ax=ax, ylim=(0, 1000))
df_4329['outbound.jitter'].rename('4329-RTT').plot(ax=ax, ylim=(0, 1000))

ax.set_title('outbound jitter over time')
ax.set_ylabel('outbound jitter[ms]')
ax.grid(True, which='both', linestyle='--', alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(40, 10))

df_D2['roundtrip.avg'].rename('D2-RTT').plot(ax=ax, ylim=(0, 1000))
df_DTAG['roundtrip.avg'].rename('DTAG-RTT').plot(ax=ax, ylim=(0, 1000))
df_Starlink['roundtrip.avg'].rename('Starlink-RTT').plot(ax=ax, ylim=(0, 1000))
df_0C['roundtrip.avg'].rename('0C-RTT').plot(ax=ax, ylim=(0, 1000))
df_4329['roundtrip.avg'].rename('4329-RTT').plot(ax=ax, ylim=(0, 1000))

ax.set_title('RTT over time')
ax.set_ylabel('RTT [ms]')
ax.grid(True, which='both', linestyle='--', alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(40, 10))

df_D2['roundtrip.jitter'].rename('D2-RTT').plot(ax=ax, ylim=(0, 1000))
df_DTAG['roundtrip.jitter'].rename('DTAG-RTT').plot(ax=ax, ylim=(0, 1000))
df_Starlink['roundtrip.jitter'].rename('Starlink-RTT').plot(ax=ax, ylim=(0, 1000))
df_0C['roundtrip.jitter'].rename('0C-RTT').plot(ax=ax, ylim=(0, 1000))
df_4329['roundtrip.jitter'].rename('4329-RTT').plot(ax=ax, ylim=(0, 1000))

ax.set_title('RTT jitter over time')
ax.set_ylabel('RTT jitter [ms]')
ax.grid(True, which='both', linestyle='--', alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(40, 10))

df_D2['roundtrip.loss'].rename('D2-RTT').plot(ax=ax, ylim=(0, 100))
df_DTAG['roundtrip.loss'].rename('DTAG-RTT').plot(ax=ax, ylim=(0, 100))
df_Starlink['roundtrip.loss'].rename('Starlink-RTT').plot(ax=ax, ylim=(0, 100))
df_0C['roundtrip.loss'].rename('0C-RTT').plot(ax=ax, ylim=(0, 100))
df_4329['roundtrip.loss'].rename('4329-RTT').plot(ax=ax, ylim=(0, 100))
ax.set_title('RTT loss over time')
ax.set_ylabel('RTT loss [%]')
ax.grid(True, which='both', linestyle='--', alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Histogram for Tx rate distributions
fig, ax = plt.subplots(figsize=(4,3))

# Choose common bins (adjust range/bins as you like)
bins = np.linspace(0, 1000, 401)  # 0–100 ms, 0.25 ms steps

ax.hist(
    df_Starlink['roundtrip.avg'].dropna(),
    bins=bins,
    alpha=0.5,
    color='red',
    #histtype='step',
    label='LEO'
)

ax.hist(
    df_D2['roundtrip.avg'].dropna(),
    bins=bins,
    alpha=0.5,
    color='green',
    #histtype='step',
    label='5G SA'
)

ax.hist(
    df_DTAG['roundtrip.avg'].dropna(),
    bins=bins,
    alpha=0.5,
    color='blue',
    #histtype='step',
    label='5G NSA'
)

ax.axvline(df_Starlink['roundtrip.avg'].mean(), linestyle='--', alpha=0.7, color='red')#, label='LEO Mean')
ax.axvline(df_D2['roundtrip.avg'].mean(), linestyle=':',  alpha=0.7, color='green')#, label='5G SA Mean')
ax.axvline(df_DTAG['roundtrip.avg'].mean(), linestyle='-.',  alpha=0.7, color='blue')#, label='5G NSA Mean')

plt.xlim(0, 100)
#ax.set_title('Histogram of RTT')
ax.set_xlabel('RTT [ms]')
ax.set_ylabel('Count')
ax.grid(True, linestyle='--', alpha=0.3)
ax.legend(loc='upper left')
plt.tight_layout()
plt.savefig("plots/TWAMP-RTT-comparison.pdf", bbox_inches="tight")
plt.savefig("plots/TWAMP-RTT-comparison.png", bbox_inches="tight")
plt.show()